In [1]:

''' 
————————————————不同训练方式的区别————————————————

训练方法    计算误差的答案   反馈的计算方式

[A]
sft        由人类生成       数学公式计算分数（交叉熵）
模型蒸馏    由模型生成       数学公式计算分数（交叉熵）

[B]
RL         无              按游戏规则打分（步骤分+结果分）
PPO        无              按游戏规则打分（（步骤分+结果分）*上一轮答题的变化幅度系数））

[C]
rlhf       无              由人类或模型打分(1-10)
DPO        无              由人类或模型打分(1 or 0)

A[通过严格输入输出训练,提高输出固定格式,负责记忆能力]
B[只针对结果和格式打分,不抠过程和细节,从而提高创造能力,泛化能力,适合开放性创造性场景]
C[由一个复杂规则打分（某种偏好,无法用具体规则描述）,捕捉到不同输出中说不出来的细微差异,
从而对齐某种偏好,（如审美,做事风格）,最终提高整体的输出质量]

备注:
[A]低质量的sft往往叫做预训练,只能学习到语言的分布规律。
[B][C]都属于强化学习。
'''

''' 
————————————————deepseek r1 训练流程————————————————

1 [RL]
使用某一领域的专业数据（数学）去训练模型的 输出格式+答案正确性
- 选择deepseek v3 作为底座模型
- 训练输出格式[sft] -> 为了通过代码批量识别输出答案的位置。(可跳过，如果跳过，模型会在下一步中学习输出格式，会有一段时间处于摸索输出格式的无效训练时期)
- 训练结果正确性[RL] -> 提高正确率。
- 总打分 = 格式分 + 正确性分。(类似于数学考试中,步骤分+答案分)
- [RL]训练 -> deepseek r1-zero

2 [生成数据]
通过泛化能力去生成大量的 其他领域cot思维链数据(推理格式+推理答案)
(成本要远低于人类生成的数据)
- deepseek r1-zero 生成 [60万条] 并批量矫正 -> [cot推理数据]
- deepseek v3 生成 [20万条] -> [非推理数据]


3 [sft]
用上一步模型生成的数据 训练出 一个新模型
- [sft]训练zero -> deepseek r1 模型

备注:
[RL]阶段关键在于,激活模型的泛化能力,解锁 模型利用本身知识 生成 其他领域的cot格式数据 的能力。
[生成数据]阶段关键在于,如何在 格式 中 自动矫正数据,从而保证数据集质量。（人类数据肯定最好，但也更贵）
[sft],数据决定模型上限,使用人类数据上限是人类,模型蒸馏其上限是原始模型。

<<整个流程的核心在于: 如何通过 格式化输出 和 代码矫正,从而获得大量 低成本 高质量的 数据。即[RL]阶段的训练。>>

'''

''' 
————————————————[RL]阶段复现成本————————————————

基座模型:qwen2.5-0.5b  约1.1g大小
推理数据集: openai/gsm8k (全数学题)
训练配置:8000条数据 累积4次更新1次 = 约2000 step
显卡类型:h20 96g
显存占用:23.430g
训练时间:5h
开始有效学习步数:92(奖励值≠0)


基座模型:qwen2.5-1.5b 约3.4g大小
推理数据集: openai/gsm8k (全数学题)
训练配置:8000条数据 累积4次更新1次 = 约2000 step
显卡类型:h20 96g
显存占用:45.4g
训练时间:6.5h
开始有效学习步数:63(奖励值≠0)

'''

'''
————————————————关于LLM的逻辑推理能力的来源————————————————

——————1 智能的定义——————
知识 = [等式]  (a=b,b=c)
逻辑 = [组合等式] (因为a=b,b=c,那么a=c)

智能 = n[等式] + n²[组合等式]
想象一下如果你知道海量的基础知识,然后你也可以充分连接这些基础知识,那么你的知识面和推理能力将非常nb。
这种组合等式的数量是全连接网络结构,将以n²的速度增长。


——————2 LLM 训练流程 ≈ 人类学习的过程——————


首先来观察主流模型的具体实现路径
预训练 -> SFT -> RLHF(PPO)      [ChatGPT, Claude, Grok, LLaMA-2]
预训练 -> SFT -> DPO            [Qwen-2, ChatGLM, DeepSeek-Chat]
预训练 -> SFT -> RLHF           [Gemini]

其实本质可以分为两个阶段
[学习等式]阶段 = 预训练 -> sft
[运用等式]阶段 = 强化学习(RLHF,DPO,PPO)
只不过llm运用 [问答对] 这样的 等价关系 进行学习 

(1) [学习等式]阶段
低质量sft(预训练) -> 高质量sft(sft)
sft 的作用 在于 记忆大量 问答对 中的 [等式]即等价关系

(2) [运用等式]阶段
强化学习(RLHF,DPO,PPO)
强化学习 的作用 在于 让模型学习如何[组合等式]
从而提高 模型的 泛化能力,创造力,输出品质等。即涌现能力。

——————3 所以该如何训练出最好的模型呢?——————
[学习等式]阶段
尽量使用严格的等价关系,
提问中,尽量说明问题的前提条件
回答中,尽量说明问题的推理过程
代码，数学等严格的语言优先

[运用等式]阶段
多种强化学习方法补齐多维度能力

——————4 为什么LLM没有创造能力——————
因为模型 没有 
创造 等式关系
验证 等式关系 
的 能力

——————5 如何将一整套理论教给ai——————
假如我有一套已经成为体系的理论框架，不需要ai自己去学习如何运用等式了，我该如何教会ai。

学习过程
1 将知识人工拆分为包含的上下几个层级
2 直到拆解到每个层级的最小知识单元
3 通过sft 让模型 学习 [最小知识单元] 和 [知识层级关系]
4 构建海量应用案例
将整体与局部分类
输入=整体 任务的进度管理,任务目标,需要输入的条件
输出=局部 当前场景需要的运算内容
5 强化学习 仅仅 用于提高输出质量（因为只为了知识迁移,而不是知识发现）



——————6 未来的发展方向——————
1 如何提升等式质量？(问答对)
知识表示格式优化
更严格的知识表示方式,如知识图谱,元数据等强等式关系

2 如何提升模型组合等式的能力？
在推理中途使用外部工具的能力

3 如何提升模型验证等式的能力？
结合[符号]的推理系统,符号是一切逻辑的基础

4 如何提升模型创造等式的能力？
生成假设机制
假设检验机制

5 如何提高可控性？
精确控制知识范围的推理，可动态添加删除可使用的知识范围
训练内容为
输入条件和目的,输出推理过程
'''



' 复现成本\n\n基座模型 qwen 0.5b\n推理数据集 openai/gsm8k (全数学题)\n8000 样本 累积 4 个样本 更新 1 次 = 2000 step\n\na100 40g\n27g显存\n4-5 小时\n'